In [1]:
import os
import re
import tensorflow as tf
from keras import layers
from keras.layers import TextVectorization

In [2]:
name, url = "Alice's Adventures in Wonderland", 'https://www.gutenberg.org/files/11/11-0.txt'
filepath = tf.keras.utils.get_file(f'{name}.txt', origin=url)

154573/154573 ━━━━━━━━━━━━━━━━━━━━ 1s 4us/step


In [3]:
# Чтение текста из файла и очистка
with open(filepath, encoding='utf-8') as f:
    text = f.read()[10000:]  # Пропуск первых 10,000 символов

text = re.sub(r"[\"\`\'\“\”\_\(\)]", r"", text)  # Удаление нежелательных символов
text = re.sub(r"\s+", " ", text)  # Удаление лишних пробелов

# Разделение текста на слова
text_list = [word.strip() for word in text.split(' ') if word.strip()]
print(f"Количество слов в тексте: {len(text_list)}")

# Векторизация текста
vectorize_layer = TextVectorization(output_mode="int")
vectorize_layer.adapt(text)

# Получение словаря (уникальных слов)
vocab = vectorize_layer.get_vocabulary()
print(f"Количество уникальных слов: {len(vocab)}")

Количество слов в тексте: 24674
Количество уникальных слов: 2799


In [4]:
# Константы для подготовки данных
BATCH_SIZE = 64  # Увеличение размера батча для ускорения обучения
BUFFER_SIZE = 512  # Увеличение буфера для перемешивания
MAX_LEN = 100

In [5]:
# Функция предобработки текстовых данных
def preprocess(text_l, length=MAX_LEN, batch_size=BATCH_SIZE):
    def preprocess_text(text):
        return text[:, :-1], text[:, 1:]

    return (
        tf.data.Dataset.from_tensor_slices(text_l)
        .window(length + 1, shift=5, drop_remainder=True)
        .flat_map(lambda window_ds: window_ds.batch(length + 1))
        .shuffle(BUFFER_SIZE)
        .batch(batch_size)
        .map(preprocess_text)
        .prefetch(tf.data.AUTOTUNE)
    )

In [6]:
# Разделение данных на обучающие и валидационные
split_idx = len(text_list) * 3 // 4
enc_train = vectorize_layer(" ".join(text_list[:split_idx]))
enc_valid = vectorize_layer(" ".join(text_list[split_idx:]))

# Создание датасетов
dataset_train = preprocess(enc_train)
dataset_valid = preprocess(enc_valid)

In [7]:
# Построение модели
model = tf.keras.Sequential([
    layers.Embedding(input_dim=len(vocab), output_dim=32),  # Увеличение размерности эмбеддинга
    layers.GRU(256, return_sequences=True),  # Увеличение размера GRU слоя для улучшения обучения
    layers.Dense(len(vocab), activation="softmax")
])

# Вывод структуры модели
model.summary()

# Компиляция модели
model.compile(optimizer='nadam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Настройка сохранения весов во время обучения
checkpoint_dir = './training_checkpoints'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, "checkpoint{epoch}.weights.h5"),
    save_weights_only=True
)

In [9]:
# Обучение модели
history = model.fit(
    dataset_train,
    epochs=5,
    validation_data=dataset_valid,
    callbacks=[checkpoint_callback]
)

Epoch 1/5
     58/Unknown 72s 1s/step - accuracy: 0.0400 - loss: 6.8938

c:\Users\YasperMoglot\Desktop\Ycheba\4course\4-course-ai\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


58/58 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.0402 - loss: 6.8860 - val_accuracy: 0.0842 - val_loss: 6.7273
Epoch 2/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.0514 - loss: 6.2477 - val_accuracy: 0.0842 - val_loss: 6.7803
Epoch 3/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.0532 - loss: 6.2288 - val_accuracy: 0.0827 - val_loss: 6.7598
Epoch 4/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.0673 - loss: 6.1223 - val_accuracy: 0.0909 - val_loss: 6.6623
Epoch 5/5
58/58 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.0777 - loss: 5.9093 - val_accuracy: 0.0967 - val_loss: 6.4423


In [10]:
# Генерация текста
# Функция предсказания следующего слова
def next_word(text, temperature=1):
    y_proba = model.predict(vectorize_layer([text]), verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    w_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return vocab[w_id.numpy()]

# Функция генерации текста заданной длины
def extend_text(text, n_words=20, temperature=1):
    for _ in range(n_words):
        w = next_word(text, temperature)
        text += f" {w}"
    return text

# Пример генерации текста
start_text = "Alice was"
generated_text = extend_text(start_text, n_words=20)
print(f"Сгенерированный текст: {generated_text}")

Сгенерированный текст: Alice was trying and five venture to say went and doesn’t chimneys thoughtfully else no looking at last said the less what
